In [1]:
import processing
train_dataloader, test_dataloader = processing.get_train_test_dataloaders('..\\dataset\\np_dataset')

In [2]:
import models.xlstm
import train 
xlstm_dict = train.get_xlstm_dict()
model = models.xlstm.xLSTM(xlstm_dict)
for batch_idx, (src, trg, metadata) in enumerate(train_dataloader):
    output = model(src.float())

In [7]:
import os
os.makedirs('pretrained/mamba/thinginin.pth')

'2025-03-06-12-52-32'

In [15]:
import torch
import os
import train
import models
xlstm_dict = train.get_xlstm_dict()

model = models.xlstm.xLSTM(xlstm_dict)

def save_model(model, path):
    if not os.path.exists(path):
        os.makedirs(os.path.dirname(path))
    torch.save(model.state_dict(), path)
save_model(model, 'pretrained/mamba/thinginin.pth')